In [2]:
import os
import zipfile
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile
import kaggle
import datetime
import re
import sklearn
import pandas as pd

/var/folders/7y/x1v97r256dx7xn5t5tq8lkg00000gn/T/ipykernel_39307/4268079309.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [37]:
dataframes = {}
download_dir = "./data"

for filename in os.listdir(download_dir):
    if filename.endswith(".csv"):
        df_name = os.path.splitext(filename)[0]  # Extract DataFrame name from filename
        df = pd.read_csv(os.path.join(download_dir, filename))
        dataframes[df_name] = df

In [33]:
players_df = dataframes['players']
players_df['birthDate'] = pd.to_datetime(players_df['birthDate'])

ValueError: time data "03/10/1992" doesn't match format "%Y-%m-%d", at position 261. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [28]:
players_df['birthDate'] = pd.to_datetime(players_df['birthDate'])
players_df['age'] = (pd.Timestamp.now().normalize() - players_df['birthDate']).dt.days // 365

In [38]:
dataframes['players']

,nflId,height,weight,birthDate,collegeName,position,displayName
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady
1,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters
2,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers
3,30842,6-6,267,1984-05-19,UCLA,TE,Marcedes Lewis
4,33084,6-4,217,1985-05-17,Boston College,QB,Matt Ryan
...,...,...,...,...,...,...,...
1678,55200,6-6,266,NaN,Indiana,DT,Ryder Anderson
1679,55212,6-0,230,NaN,Iowa State,ILB,Jake Hummel
1680,55239,6-2,300,NaN,Pennsylvania,DT,Prince Emili
1681,55240,6-1,185,NaN,Buffalo,CB,Ja'Marcus Ingram


In [17]:
fake_date = "02/69/1993"

fake_date = re.sub(r'(\d{2})/(\d{2})/(\d{4})', r'\3-\1-\2', fake_date)

pattern = r'^\d{4}-\d{2}-\d{2}$'
if not re.match(pattern, fake_date):
    print('yes')

re.match(pattern, fake_date)

<re.Match object; span=(0, 10), match='1993-02-69'>

In [39]:
pattern = r'^\d{4}-\d{2}-\d{2}$'
for i, date in enumerate(players_df['birthDate']):
    if not pd.isna(date):
        if not re.match(pattern, str(date)[0:10]):
            players_df.loc[i, 'birthDate'] = re.sub(r'(\d{2})/(\d{2})/(\d{4})', r'\3-\1-\2', date)
            print(players_df.loc[i, 'birthDate'])

players_df['birthDate'] = pd.to_datetime(players_df['birthDate'])
players_df['age'] = (pd.Timestamp.now().normalize() - players_df['birthDate']).dt.days // 365
players_df

,nflId,height,weight,birthDate,collegeName,position,displayName,age
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady,46.0
1,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters,42.0
2,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers,40.0
3,30842,6-6,267,1984-05-19,UCLA,TE,Marcedes Lewis,39.0
4,33084,6-4,217,1985-05-17,Boston College,QB,Matt Ryan,38.0
...,...,...,...,...,...,...,...,...
1678,55200,6-6,266,NaT,Indiana,DT,Ryder Anderson,NaN
1679,55212,6-0,230,NaT,Iowa State,ILB,Jake Hummel,NaN
1680,55239,6-2,300,NaT,Pennsylvania,DT,Prince Emili,NaN
1681,55240,6-1,185,NaT,Buffalo,CB,Ja'Marcus Ingram,NaN


In [41]:
players_df['height']
players_df['height_inches'] = players_df['height'].apply(lambda x: int(x.split('-')[0]) * 12 + int(x.split('-')[1]))
players_df

,nflId,height,weight,birthDate,collegeName,position,displayName,age,height_inches
0,25511,6-4,225,1977-08-03,Michigan,QB,Tom Brady,46.0,76
1,29550,6-4,328,1982-01-22,Arkansas,T,Jason Peters,42.0,76
2,29851,6-2,225,1983-12-02,California,QB,Aaron Rodgers,40.0,74
3,30842,6-6,267,1984-05-19,UCLA,TE,Marcedes Lewis,39.0,78
4,33084,6-4,217,1985-05-17,Boston College,QB,Matt Ryan,38.0,76
...,...,...,...,...,...,...,...,...,...
1678,55200,6-6,266,NaT,Indiana,DT,Ryder Anderson,NaN,78
1679,55212,6-0,230,NaT,Iowa State,ILB,Jake Hummel,NaN,72
1680,55239,6-2,300,NaT,Pennsylvania,DT,Prince Emili,NaN,74
1681,55240,6-1,185,NaT,Buffalo,CB,Ja'Marcus Ingram,NaN,73


In [59]:
tracking_drop_cols = dataframes['tracking_week_1']
players_drop_cols = players_df

merged_df = pd.merge(tracking_drop_cols, players_drop_cols[['nflId', 'weight', 'age', 'height_inches']], on='nflId', how='left')

In [60]:
tackles_df = dataframes['tackles']
all_merged = pd.merge(merged_df, tackles_df, on=['gameId', 'playId', 'nflId'], how='left')

In [61]:
all_merged.columns

Index(['gameId', 'playId', 'nflId', 'displayName', 'frameId', 'time',
       'jerseyNumber', 'club', 'playDirection', 'x', 'y', 's', 'a', 'dis', 'o',
       'dir', 'event', 'weight', 'age', 'height_inches', 'tackle', 'assist',
       'forcedFumble', 'pff_missedTackle'],
      dtype='object')

In [58]:
players_drop_cols.columns

Index(['nflId', 'height', 'weight', 'birthDate', 'collegeName', 'position',
       'displayName', 'age', 'height_inches'],
      dtype='object')

In [63]:
ml_df = pd.read_csv('/Users/pranavkirti/Desktop/College/Spring 2024/CSE 314/final_314/data/final_nfl_dataframe.csv')
ml_df

,Unnamed: 0,gameId,playId,nflId,x,y,s,a,dis,o,dir,Week,weight,weight_kg,force,tackle,assist,forcedFumble,pff_missedTackle
0,132,2022090800,56,43294.0,77.95,40.78,5.50,3.00,0.54,139.54,172.50,1,208.0,94.347136,283.041408,1.0,0.0,0.0,0.0
1,133,2022090800,56,43294.0,78.02,40.21,5.76,2.29,0.56,140.14,173.60,1,208.0,94.347136,216.054941,1.0,0.0,0.0,0.0
2,134,2022090800,56,43294.0,78.06,39.63,5.97,1.78,0.59,143.65,175.43,1,208.0,94.347136,167.937902,1.0,0.0,0.0,0.0
3,135,2022090800,56,43294.0,78.10,39.03,6.02,1.18,0.60,143.65,176.65,1,208.0,94.347136,111.329620,1.0,0.0,0.0,0.0
4,136,2022090800,56,43294.0,78.13,38.44,5.99,0.97,0.60,141.83,177.73,1,208.0,94.347136,91.516722,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804154,12167083,2022110700,3787,52578.0,24.37,19.39,1.98,2.21,0.17,64.39,137.50,9,305.0,138.345560,305.743688,1.0,0.0,0.0,0.0
804155,12167084,2022110700,3787,52578.0,24.46,19.28,1.64,2.69,0.13,65.07,140.24,9,305.0,138.345560,372.149556,1.0,0.0,0.0,0.0
804156,12167085,2022110700,3787,52578.0,24.55,19.16,1.39,2.69,0.15,69.25,142.09,9,305.0,138.345560,372.149556,1.0,0.0,0.0,0.0
804157,12167086,2022110700,3787,52578.0,24.59,19.13,1.02,2.30,0.05,77.87,142.60,9,305.0,138.345560,318.194788,1.0,0.0,0.0,0.0


In [102]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

X = ml_df[['x', 'y', 's', 'a', 'dis', 'o', 'dir', 'weight_kg', 'force']]
y = ml_df['forcedFumble']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

logistic_regression = LogisticRegression()
logistic_regression.fit(X_train, y_train)
y_pred = logistic_regression.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9958528153601274


In [99]:
y_zero_rows = ml_df[ml_df['forcedFumble'] == 0]
y_one_rows = ml_df[ml_df['forcedFumble'] == 1]

num_y_zero = len(y_zero_rows)
num_y_one = len(y_one_rows)

y_zero_rows_downsampled = y_zero_rows.sample(n=num_y_one)
balanced_df = pd.concat([y_zero_rows_downsampled, y_one_rows])
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

In [105]:
X = balanced_df[['x', 'y', 's', 'a', 'dis', 'o', 'dir', 'weight_kg', 'force']]
y = balanced_df['forcedFumble']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

logistic_regression = LogisticRegression()
logistic_regression.fit(X_train, y_train)
y_pred = logistic_regression.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5951184493898062


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9490308686288585


In [115]:
results = {'accuracy': accuracy,
              'conf matrix': conf_matrix.tolist()}

In [117]:
import json

os.makedirs('./model_results', exist_ok=True)
results_path = './model_results/randomforest_results.json'
with open(results_path, "w") as json_file:
    json.dump(results, json_file)

In [118]:
len(os.listdir("./data"))

1

In [15]:
final_nfl_dataframe = pd.read_csv("./data/final_nfl_dataframe.csv")  # Replace with actual path
   
# Check if all PlayIDs in the final dataframe are present in tackles data
tackles_data = pd.read_csv("./data/tackles.csv")  # Load tackles data
tackles_data = tackles_data[tackles_data['tackle'] == 1]
tackles_playids = set(tackles_data['playId'].unique())
final_nfl_dataframe_playids = set(final_nfl_dataframe['playId'].unique())

In [16]:
len(tackles_playids)

3943

In [13]:
len(final_nfl_dataframe_playids)

1704

In [11]:
tackles_data

,gameId,playId,nflId,tackle,assist,forcedFumble,pff_missedTackle
0,2022090800,101,42816,1,0,0,0
1,2022090800,393,46232,1,0,0,0
2,2022090800,486,40166,1,0,0,0
3,2022090800,646,47939,1,0,0,0
4,2022090800,818,40107,1,0,0,0
...,...,...,...,...,...,...,...
17421,2022091113,2494,43533,0,0,0,1
17422,2022092502,3510,42406,0,0,0,1
17423,2022091113,3642,43478,0,0,0,1
17424,2022091901,3578,42431,0,0,0,1
